In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os, shutil
from tqdm import tqdm
import pickle
import math

from keras_applications.resnet import ResNet152
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten,Input, concatenate
from keras.models import Model, Sequential, load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend, layers, models,utils
from keras.utils import multi_gpu_model
from keras_efficientnets.efficientnet import EfficientNetB3

import numpy as np
import pandas as pd
from collections import Counter
import tensorflow as tf
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
print(device_lib.list_local_devices())

为每个版本设编号

In [ ]:
ID = 0

## 预处理

去除异常值并检查样本分布。

In [ ]:
with open('ex_files.txt', 'rb') as ef:
    ex_files = pickle.load(ef)
data_dir = './{}/'
files = {x: os.listdir(data_dir.format(x)) for x in ['train', 'test']}
clean_data = [i for i in files['train'] if i not in ex_files['resnet50']]
clean_labels = [x.split('.')[0] for x in clean_data]
df_smpl = pd.DataFrame({i: clean_labels.count(i) for i in set(clean_labels)}, index=[0])

plt.pie(df_smpl, labels = df_smpl.columns, startangle=90, autopct = '%3.1f%%', textprops={'fontsize':16});
plt.title('Sample size distribution', fontsize = 20)
plt.savefig('sample_size_distribution_train.jpg')

切分训练集和验证集。

In [ ]:
data_dir = './train/'
test_size = 0.2

with open('data_set.txt', 'rb') as ds:
    data_set = pickle.load(ds)
file_df = pd.DataFrame({'file_names': clean_data, 'labels': [x.split('.')[0] for x in clean_data]})
file_train, file_valid, label_train, label_valid = train_test_split(file_df['file_names'], file_df['labels'],
                                                                    test_size = test_size, random_state = 0)
file_valid = file_valid.tolist()
file_train = file_train.tolist()
print(len(file_train), len(file_valid), len(label_train), len(label_valid))

In [ ]:
data_set = {'file_train': file_train, 'label_train': label_train, 'file_valid': file_valid, 'label_valid': label_valid}
with open('data_set.txt', 'wb') as ds:
    pickle.dump(data_set, ds)

检查训练集和验证集样本分布。

In [ ]:
plt.figure(figsize=[20, 6])
plt.subplot(1, 2, 1)
train_dist = pd.DataFrame(Counter(label_train), index=[0])
ax, ltext, ptext = plt.pie(train_dist, labels = train_dist.columns, startangle=90, autopct = '%3.1f%%', textprops={'fontsize':16});
plt.title('Sample size distribution of training set', fontsize = 20)

plt.subplot(1, 2, 2)
valid_dist = pd.DataFrame(Counter(label_valid), index=[0])
plt.pie(valid_dist, labels = valid_dist.columns, startangle=90, autopct = '%3.1f%%', textprops={'fontsize':16})
plt.title('Sample size distribution of validation set', fontsize = 20);

plt.savefig('sample_size_distribution_train_valid.jpg')

In [ ]:
dataf = {x: pd.DataFrame({'X': data_set['file_{}'.format(x)],
                          'Y'.format(x): data_set['label_{}'.format(x)]}) for x in ['train', 'valid']}
dataf['train'].head()

数据增强。

In [ ]:
datagen = {x: image.ImageDataGenerator(preprocessing_function=preprocess_input) for x in dataf.keys()}
# datagen['train'].horizontal_flip = True
# datagen['train'].rotation_range = 30
# datagen['train'].zoom_range = 0.2
# datagen['train'].vertical_flip = True
# datagen['train'].zoom_range

In [ ]:
im_width, im_height = 224, 224
batch_size = 128

generator = {x: datagen[x].flow_from_dataframe(
    dataf[x], x_col = 'X', y_col = 'Y',directory = data_dir.format('train'),
    target_size=(im_width, im_height),
    batch_size=batch_size,
    seed = 123,
    class_mode = 'binary'
) for x in datagen.keys()}

generator['valid'].shuffle = False

## 构建模型

In [ ]:
model_name = 'ResNet152'
model_base = ResNet152(weights = 'imagenet', include_top = False, input_shape = (im_width, im_height, 3),
                           backend = backend, layers = layers, models = models, utils = utils)
model = Sequential()
model.add(model_base)
model.add(GlobalAveragePooling2D())
# model.add(Dropout(0.75))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
print('Number of trainable weights befor freezing the model_base:', len(model.trainable_weights))
model_base.trainable = False
print('Number of trainable weights after freezing the model_base:', len(model.trainable_weights))

## 训练模型

In [ ]:
lr = 0.00005
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss = "binary_crossentropy", optimizer = optimizers.Adam(lr=lr), metrics=["accuracy"])

epochs = 40
history = model.fit_generator(generator['train'],
                              steps_per_epoch=math.ceil(generator['train'].samples / batch_size),
                              epochs=epochs,
                              validation_data=generator['valid'],
                              validation_steps=math.ceil(generator['valid'].samples / batch_size),
                              callbacks=[early_stopping],
                              verbose = 2)
model.save('models/model_{}.h5'.format(ID))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

x_axis = range(1, len(acc)+1)

plt.figure(figsize = [20, 6])
plt.subplots_adjust(wspace=0.3)
plt.subplot(1,2,1)
plt.plot(x_axis, acc, 'blue', label='Training Accurarcy')
plt.plot(x_axis, val_acc, 'red', label='Validation Accurarcy')
plt.title('Training and Validation Accurarcy', fontsize = 20)
plt.xlabel('Epoches', fontsize = 16)
plt.ylabel('Accuracy', fontsize = 16)
plt.legend(fontsize = 14)
plt.xticks(ticks=range(0, len(acc)+1, 2),fontsize = 12)
plt.yticks(fontsize = 12)


plt.subplot(1,2,2)
plt.plot(x_axis, loss, 'blue', label='Training Loss')
plt.plot(x_axis, val_loss, 'red', label='Validation Loss')
plt.title('Training and Validation Loss', fontsize = 20)
plt.xlabel('Epoches', fontsize = 16)
plt.ylabel('Loss (Binary cross entropy)', fontsize = 16)
plt.legend(fontsize = 14)
plt.xticks(ticks=range(0, len(acc)+1, 2), fontsize = 12)
plt.yticks(fontsize = 12)

# plt.suptitle('Pre-trained model: {}\nLearning rate: {}'.format(model_name, lr),
#             x = 0.1, y = 1, ha = 'left', fontsize = 14)
plt.savefig('results/Loss and Accuracy {}.jpg'.format(ID))

## 预测

In [ ]:
test_imgs = os.listdir('test/')
ids = []
label = []
for i in tqdm(test_imgs):
    img = image.load_img('test/' + i, target_size=(im_width, im_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    label.append(model.predict(x)[0][0])
    ids.append(int(i.split('.')[0]))
sub = pd.DataFrame({'id': ids, 'label': label}).sort_values('id',axis = 0, ascending = True)
sub.to_csv('results/submission {}.csv'.format(ID), index = False)

In [ ]:
with open('models/history {}.txt'.format(ID), 'wb') as his:
    pickle.dump(history, his)

In [ ]:
test_imgs = os.listdir(data_dir.format('test'))
img_path = data_dir.format('test') + test_imgs[3]
# img_path = './test04.jpg'
img = image.load_img(img_path, target_size=(im_width, im_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)[0][0]
print(preds)
img_show = mpimg.imread(img_path)
plt.imshow(img_show)
plt.suptitle('This is a {}'.format('dog' if preds > 0.5 else 'cat'))
plt.title('probability: {}%'.format(round(preds*100, 2)) if preds > 0.5 else round((1-preds)*100, 2));